# Dot product used by CUDA and OpenMP

* 在NVIDIA_CUDA-8.0_Samples/0_Simple/UnifiedMemoryStreams有使用CUDA和OpenMP的編譯方法，所以根據這個Makefile進行解碼，再把網路上有使用CUDA和OpenMP解Dot product的code在我的MAC進行compile以及execute，可惜OpenMP部分OS X沒有支援

* 因為OS X 的xcode沒有支援-Xcompiler -fopenmp，所以註解掉main.cu的`include <omp.h>`就可以執行

  [原因](http://docs.nvidia.com/cuda/cuda-samples/index.html#cudaopenmp)
>"cudaOpenMP的Supported OSes只有Linux, Windows而且sample只有MAC沒有cudaOpenMP資料夾"

* 因為CUDA的原始Makefile會在/Users/chiustin/NVIDIA_CUDA-8.0_Samples開一個bin資料夾放執行檔，所以在Makefile裡也在這兩行做註解   
  - `$(EXEC) mkdir -p ../../bin/$(TARGET_ARCH)/$(TARGET_OS)/$(BUILD_TYPE)` 
  - `$(EXEC) cp $@ ../../bin/$(TARGET_ARCH)/$(TARGET_OS)/$(BUILD_TYPE)`

## compile分成兩個部分進行解碼
`main.o:main.cu
	$(EXEC) $(NVCC) $(INCLUDES) $(ALL_CCFLAGS) $(GENCODE_FLAGS) -o $@ -c $<`

`main: main.o
	$(EXEC) $(NVCC) $(ALL_LDFLAGS) $(GENCODE_FLAGS) -o $@ $+ $(LIBRARIES)`

### main.o:main.cu
* `EXEC ?= @echo "[@]"`
* `NVCC := $(CUDA_PATH)/bin/nvcc -ccbin $(HOST_COMPILER)`
    - `CUDA_PATH?=/Developer/NVIDIA/CUDA-8.0`
    - `HOST_COMPILER ?= clang++`
        * `ifeq ($(TARGET_OS),darwin)` #在OS X CUDA是開啟darwin資料夾
* `INCLUDES  := -I/Users/chiustin/NVIDIA_CUDA-8.0_Samples/common/inc` #helper_cuda.h
* `ALL_CCFLAGS`
  - `ALL_CCFLAGS += $(NVCCFLAGS)`
      - `NVCCFLAGS   := -m${TARGET_SIZE}`
          - `ifneq (,$(filter $(TARGET_ARCH),x86_64 aarch64 ppc64le))
            TARGET_SIZE := 64` #OS X是x86_64
  - `ALL_CCFLAGS += $(EXTRA_NVCCFLAGS)`
  - `ALL_CCFLAGS += $(addprefix -Xcompiler ,$(CCFLAGS))` #addprefix會讓Xcompiler分-arch和HOST_ARCH
      - `-arch $(HOST_ARCH)`
          - `HOST_ARCH   := $(shell uname -m)` #在OS X uname -m是x86_64
  - `ALL_CCFLAGS += $(addprefix -Xcompiler ,$(EXTRA_CCFLAGS))`
  - else #算有點小BUG，因為OS X是判斷`ifneq ($(TARGET_OS),darwin)`，pthread才是判斷else
    - `LIBRARIES += -lpthread`
    - `ALL_CCFLAGS += -DUSE_PTHREADS` #因為xcode不支援-fopenmp
* GENCODE_FLAGS #分成SASS和PTX後面會進行解釋
  - `GENCODE_FLAGS += -gencode arch=compute_$(sm),code=sm_$(sm)`
      - `$(foreach sm,$(SMS),$(eval GENCODE_FLAGS += -gencode arch=compute_$(sm),code=sm_$(sm)))`
          - `SMS ?= 30 35 37 50 52 60` #foreach sm會將SMS都

  - `GENCODE_FLAGS += -gencode arch=compute_$(HIGHEST_SM),code=compute_$(HIGHEST_SM)`
      - `HIGHEST_SM := $(lastword $(sort $(SMS)))` #做sort由小到大，最大的SMS=60為lastword


### main: main.o
* ALL_LDFLAGS 
  - `ALL_LDFLAGS += $(addprefix -Xlinker ,$(LDFLAGS))`
      - `LDFLAGS += -rpath $(CUDA_PATH)/lib` #lcublas
  - `ALL_LDFLAGS += $(addprefix -Xlinker ,$(EXTRA_LDFLAGS))`

## 以下是我的Command Line instrument:

/Developer/NVIDIA/CUDA-8.0/bin/nvcc -ccbin clang++ -I/Users/chiustin/NVIDIA_CUDA-8.0_Samples/common/inc  -m64  -Xcompiler -arch -Xcompiler x86_64  -DUSE_PTHREADS -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_60,code=compute_60 -o main.o -c main.cu

/Developer/NVIDIA/CUDA-8.0/bin/nvcc -ccbin clang++   -m64  -Xcompiler -arch -Xcompiler x86_64  -DUSE_PTHREADS -Xlinker -rpath -Xlinker /Developer/NVIDIA/CUDA-8.0/lib  -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_60,code=compute_60 -o main main.o  -lcublas

## 解釋SASS和PTX

NVIDIA的ParallelThread eXecution (PTX)指令集。 PTX是一種偽彙編指令集；即它不能直接運行在硬件之上。 Ptxas是NVIDIA的彙編器，將PTX彙編為可運行在硬件上本機指令集（SASS）。每一代硬件支持不同版本的SASS。出於這個原因，PTX在編譯的時候可以被編譯為面向不同代硬件的不同版本的SASS。儘管如此，PTX代碼仍然被嵌入到庫中以支持未來的硬件。在運行時，運行時系統選擇合適版本的SASS運行在相應的硬件上。如果沒有合適版本，則運行時系統調用just-in-time（JIT）編譯器編譯嵌入的PTX，將其編譯為對應於相應硬件的SASS。

## clang gcc omp.h
* /usr/local/Cellar/gcc/6.2.0/lib/gcc/6/gcc/x86_64-apple-darwin16.0.0/6.2.0/include
    - 所以gcc -fopenmp omp.c是可以產生執行檔並且執行

## cuda.h 
* /Developer/NVIDIA/CUDA-8.0/lib/stubs/CUDA.framework/Headers
* /usr/local/cuda/include